# Time Series Forecasting

In [36]:
import darts
import plotly.graph_objects as go
import polars as pl
from darts.datasets import AirPassengersDataset
from darts.utils.statistics import (
    check_seasonality,
    extract_trend_and_seasonality,
    stationarity_test_adf,
)
from plotly.express import line
from plotly.subplots import make_subplots

series: darts.TimeSeries = AirPassengersDataset().load()
dataframe: pl.DataFrame = series.to_dataframe(backend="polars", time_as_index=False)


In [37]:
line(
    x=dataframe["Month"],
    y=dataframe["#Passengers"],
    width=800,
    height=400,
    labels={"x": "Month", "y": "Passengers"},
    title="Air Passengers Over Time",
)

In [38]:
check_seasonality(series)


(True, np.int64(12))

Am I really going to believe whatever the computer says? Yes, yes I will.

In [39]:
x, y = extract_trend_and_seasonality(series)
x_df = x.to_dataframe(backend="polars", time_as_index=False)
y_df = y.to_dataframe(backend="polars", time_as_index=False)

In [40]:
fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=x_df["Month"], y=x_df["0"]),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(x=y_df["Month"], y=y_df["0"]),
    row=1,
    col=2,
)

fig.update_layout(height=400, title_text="Trend and Seasonality")
fig.show()


In [41]:
stationarity_test_adf(series)

(np.float64(0.8153688792060498),
 np.float64(0.991880243437641),
 13,
 130,
 {'1%': np.float64(-3.4816817173418295),
  '5%': np.float64(-2.8840418343195267),
  '10%': np.float64(-2.578770059171598)},
 np.float64(996.692930839019))

The uncertainty of the forecast is just as important as, or even more so, than the forecast itself.